In [90]:
import json
import random
from tqdm.notebook import trange, tqdm

In [91]:
from transformers import GPT2Tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

In [92]:
with open("ERAI Dataset_with_translation/ERAI_Dataset_pairwise_test_with_translation.json","r") as fp:
    tdata = json.load(fp)

In [93]:
with open("ERAI Dataset_with_translation/ERAI_Dataset_pairwise_train_with_translation.json","r") as fp:
    data = json.load(fp)

In [94]:
data[0].keys()

dict_keys(['index', 'post1', 'post2', 'MPP1', 'MPP2', 'ML1', 'ML2', 'MPP_label', 'ML_label', 'post1_en', 'post2_en'])

In [95]:
texts = []
for d in data:
    texts.append('post1 : ' + d['post1']+' post2: ' + d['post2']+ ' > maximal potential profit (MPP)| {}# maximal loss (ML)| {}</s> '.format(d['MPP_label'], d['ML_label']))

In [96]:
texts_lens = []
for t in texts:
    tks = tokenizer(t)
    texts_lens.append(len(tks['input_ids']))

Token indices sequence length is longer than the specified maximum sequence length for this model (1088 > 1024). Running this sequence through the model will result in indexing errors


In [97]:
def create_prompt(texts, texts_lens, q, qlen, max_len=2048, sampling_patience=3, seed=42):
    p = ""
    plen = 0
    random.seed(seed)
    sampling_tolerance = 0
    to_be_sampled = list(range(len(texts)))
    count = 0
    while True:
        # generate a candidate without replacement
        cidx = random.sample(to_be_sampled, 1)
        cidx = cidx[0]
       
        if texts_lens[cidx] + qlen + plen > max_len:
            if sampling_tolerance < sampling_patience:
                sampling_tolerance += 1
                continue
            else:
                break
                
        p += texts[cidx]
        plen += texts_lens[cidx]
        count += 1
        
    return p + q

In [108]:
prompts = []
for d in tqdm(tdata):
    q = 'post1 : ' +d['post1']+' post2: ' + d['post2']+ ' > maximal potential profit (MPP)|'
    q_en = d['post1_en']+' ' + d['post2_en']+ ' > maximal potential profit (MPP)|'
    qlen = len(tokenizer(q)['input_ids'])
    if qlen > 4000:
        q = q[0:int(7*len(q)/16)]
        qlen = len(tokenizer(q)['input_ids'])
    p = create_prompt(texts, texts_lens, q, qlen, max_len=4000, sampling_patience=10, seed=42)
    prompts.append({'index':d['index'],'q':q,'q_en':q_en,'prompt':p})

  0%|          | 0/87 [00:00<?, ?it/s]

In [ ]:
with open('prompts/openai_prompts_42.json', 'w') as f:
    json.dump(prompts, f)

In [109]:
len(tokenizer(prompts[-31]['q'])['input_ids'])

3203

In [110]:
tdata[-31]

{'index': 512,
 'post1': '生意來了，好的開始是成功的一半。\n 都是堅持做對的事，老闆娘與老闆不一樣了。\n \n 神秘金字塔-股權分散表-裕隆\n http://norway.twsthr.info/StockHolders.aspx?stock=%E8%A3%95%E9%9A%86\n \n 股狗網-籌碼分析-裕隆\n https://www.stockdog.com.tw/stockdog/index.php?m=overview&sid=2201\n \n 淳紳與裕隆簽訂合作意向書 委請整車、電池包組裝代工\n 鉅亨網記者劉韋廷 台北 2019年8月26號21點48分\n 電動車廠淳紳 (4529-TW) 今 (26) 日宣布，將與裕隆汽車 (2201-TW) 簽訂合作意向書，簽訂有效期限為 3 月，擬委託裕隆進行整車及電池包組裝代工。淳紳表示，因裕隆擁有車輛製造專業技術及能力，擬委託裕隆進行整車 SKD(semi knock down)與淳紳自主開發的電池包組裝代工，將先請裕隆進行專案執行評估及試作事宜，並經雙方協商後達成初步共識，將就相關事項簽訂意向書。淳紳表示，合作意向書有效期間為 3 個月，若本專案需較長之評估過程，雙方得合意延長本意向書之期間，而意向書僅敘述雙方合作意願與共識，對於公司財務與業務並未產生影響。\n \n \n 新店裕隆城 1080826工地空拍',
 'post2': '裕隆組織大改造！納智捷董座陳國榮辭職 由裕隆總座姚振祥兼任\n ETtoday新聞雲 記者周康玉／台北報導 2019年09月12日 17:07\n \n 裕隆（2201）集團旗下自有品牌納智捷（LUXGEN）銷售表現始終未見起色，裕隆今（12）日證實，納智捷董事長、同時身兼華創車電董座的陳國榮已主動辭職，為品牌表現不佳負責。裕隆表示，已慰留陳國榮華創車電董座職務，未來納智捷董座由裕隆汽車總經理姚振祥兼任；裕隆強調，同意陳國榮辭董座是基於策略轉型考量，陳國榮仍為集團重要的決策委員。\n \n 裕隆表示，為了落實集團汽車價值鏈轉型策略及華創車電廠辦合一，集團組織及人事進行以下調整：\n 一. 設立「自有品牌暨開放平台整合專案室」，負責納智捷品牌及開放平台價值鏈研發、採購、製造、銷售之協調與整合。該專案室總監乙職由裕隆汽車總經理姚振祥兼任，副總監

In [111]:
len(tokenizer(tdata[-31]['post1'])['input_ids'])

861

In [112]:
len(tokenizer(tdata[-31]['post2'])['input_ids'])

6828

In [119]:
prompts[-31]

{'index': 512,
 'q': 'post1 : 生意來了，好的開始是成功的一半。\n 都是堅持做對的事，老闆娘與老闆不一樣了。\n \n 神秘金字塔-股權分散表-裕隆\n http://norway.twsthr.info/StockHolders.aspx?stock=%E8%A3%95%E9%9A%86\n \n 股狗網-籌碼分析-裕隆\n https://www.stockdog.com.tw/stockdog/index.php?m=overview&sid=2201\n \n 淳紳與裕隆簽訂合作意向書 委請整車、電池包組裝代工\n 鉅亨網記者劉韋廷 台北 2019年8月26號21點48分\n 電動車廠淳紳 (4529-TW) 今 (26) 日宣布，將與裕隆汽車 (2201-TW) 簽訂合作意向書，簽訂有效期限為 3 月，擬委託裕隆進行整車及電池包組裝代工。淳紳表示，因裕隆擁有車輛製造專業技術及能力，擬委託裕隆進行整車 SKD(semi knock down)與淳紳自主開發的電池包組裝代工，將先請裕隆進行專案執行評估及試作事宜，並經雙方協商後達成初步共識，將就相關事項簽訂意向書。淳紳表示，合作意向書有效期間為 3 個月，若本專案需較長之評估過程，雙方得合意延長本意向書之期間，而意向書僅敘述雙方合作意願與共識，對於公司財務與業務並未產生影響。\n \n \n 新店裕隆城 1080826工地空拍 post2: 裕隆組織大改造！納智捷董座陳國榮辭職 由裕隆總座姚振祥兼任\n ETtoday新聞雲 記者周康玉／台北報導 2019年09月12日 17:07\n \n 裕隆（2201）集團旗下自有品牌納智捷（LUXGEN）銷售表現始終未見起色，裕隆今（12）日證實，納智捷董事長、同時身兼華創車電董座的陳國榮已主動辭職，為品牌表現不佳負責。裕隆表示，已慰留陳國榮華創車電董座職務，未來納智捷董座由裕隆汽車總經理姚振祥兼任；裕隆強調，同意陳國榮辭董座是基於策略轉型考量，陳國榮仍為集團重要的決策委員。\n \n 裕隆表示，為了落實集團汽車價值鏈轉型策略及華創車電廠辦合一，集團組織及人事進行以下調整：\n 一. 設立「自有品牌暨開放平台整合專案室」，負責納智捷品牌及開放平台價值鏈研發、採購、製造、銷售之協調與整合。該專案室總監乙職由裕隆汽車總經理姚振祥兼任，副總監由華

In [114]:
len(tokenizer(prompts[56]['q'])['input_ids'])

3203

In [115]:
tdata[56]['post2']

'裕隆組織大改造！納智捷董座陳國榮辭職 由裕隆總座姚振祥兼任\n ETtoday新聞雲 記者周康玉／台北報導 2019年09月12日 17:07\n \n 裕隆（2201）集團旗下自有品牌納智捷（LUXGEN）銷售表現始終未見起色，裕隆今（12）日證實，納智捷董事長、同時身兼華創車電董座的陳國榮已主動辭職，為品牌表現不佳負責。裕隆表示，已慰留陳國榮華創車電董座職務，未來納智捷董座由裕隆汽車總經理姚振祥兼任；裕隆強調，同意陳國榮辭董座是基於策略轉型考量，陳國榮仍為集團重要的決策委員。\n \n 裕隆表示，為了落實集團汽車價值鏈轉型策略及華創車電廠辦合一，集團組織及人事進行以下調整：\n 一. 設立「自有品牌暨開放平台整合專案室」，負責納智捷品牌及開放平台價值鏈研發、採購、製造、銷售之協調與整合。該專案室總監乙職由裕隆汽車總經理姚振祥兼任，副總監由華創車電左自生副總經理擔任。\n 二. 另為強化溝通及運作效率，「自有品牌暨開放平台整合專案室」直接向集團決策委員會報告。另，納智捷汽車董事長一職改由裕隆汽車總經理姚振祥兼任。\n 三. 華創車電總經理李俊忠辭任，改由副總經理黃振宏暫代華創車電總經理職務。\n 四. 華創車電董事長陳國榮請辭，已予慰留。\n 五. 集團總管理處與決策委員會即日起不再設置副執行長。\n \n 裕隆集團同時發出三大聲明：\n 一. 為落實裕隆汽車價值鏈轉型及華創車電廠辦合一策略，裕隆集團新成立「自有品牌暨開放平台整合專案室」，統籌研發、採購、製造、銷售等價值鏈的協調整合，以落實開放平台、多元客戶之轉型策略。\n 二. 為強化縱向溝通及管理決策效率，「自有品牌暨開放平台整合專案室」直接向集團決策委員會報告，未來廠辦合一在三義，該新設專案室總監將由裕隆汽車姚振祥總經理兼任，副總監由華創車電副總經理左自生擔任。而為進一步強化溝通效率，姚振祥將兼任台灣納智捷董事長，原董事長陳國榮卸任，同時華創車電總經理李俊忠請辭，改由副總經理黃振宏暫代。\n 三. 轉型策略需要更好的橫向及縱向溝通效率，考量組織扁平化能提昇溝通效率，故不再設置副執行長職位。\n \n \n 【獨家】震撼！裕隆集團副執行長陳國榮 請辭華創及納智捷董座\n 蘋果日報 出版時間：2019/09/12 18:09\n https://tw.appledaily.com/new/realtime/2

In [116]:
tdata[56]['post1']

'生意來了，好的開始是成功的一半。\n 都是堅持做對的事，老闆娘與老闆不一樣了。\n \n 神秘金字塔-股權分散表-裕隆\n http://norway.twsthr.info/StockHolders.aspx?stock=%E8%A3%95%E9%9A%86\n \n 股狗網-籌碼分析-裕隆\n https://www.stockdog.com.tw/stockdog/index.php?m=overview&sid=2201\n \n 淳紳與裕隆簽訂合作意向書 委請整車、電池包組裝代工\n 鉅亨網記者劉韋廷 台北 2019年8月26號21點48分\n 電動車廠淳紳 (4529-TW) 今 (26) 日宣布，將與裕隆汽車 (2201-TW) 簽訂合作意向書，簽訂有效期限為 3 月，擬委託裕隆進行整車及電池包組裝代工。淳紳表示，因裕隆擁有車輛製造專業技術及能力，擬委託裕隆進行整車 SKD(semi knock down)與淳紳自主開發的電池包組裝代工，將先請裕隆進行專案執行評估及試作事宜，並經雙方協商後達成初步共識，將就相關事項簽訂意向書。淳紳表示，合作意向書有效期間為 3 個月，若本專案需較長之評估過程，雙方得合意延長本意向書之期間，而意向書僅敘述雙方合作意願與共識，對於公司財務與業務並未產生影響。\n \n \n 新店裕隆城 1080826工地空拍'

In [117]:
text = prompts[56]['q']

In [47]:
import re
text = re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)

In [52]:
len(tokenizer(text)['input_ids'])

7600

In [50]:
text = re.sub(r'http\S+', '', text)

In [51]:
text

'生意來了，好的開始是成功的一半。\n 都是堅持做對的事，老闆娘與老闆不一樣了。\n \n 神秘金字塔-股權分散表-裕隆\n \n \n 股狗網-籌碼分析-裕隆\n \n \n 淳紳與裕隆簽訂合作意向書 委請整車、電池包組裝代工\n 鉅亨網記者劉韋廷 台北 2019年8月26號21點48分\n 電動車廠淳紳 (4529-TW) 今 (26) 日宣布，將與裕隆汽車 (2201-TW) 簽訂合作意向書，簽訂有效期限為 3 月，擬委託裕隆進行整車及電池包組裝代工。淳紳表示，因裕隆擁有車輛製造專業技術及能力，擬委託裕隆進行整車 SKD(semi knock down)與淳紳自主開發的電池包組裝代工，將先請裕隆進行專案執行評估及試作事宜，並經雙方協商後達成初步共識，將就相關事項簽訂意向書。淳紳表示，合作意向書有效期間為 3 個月，若本專案需較長之評估過程，雙方得合意延長本意向書之期間，而意向書僅敘述雙方合作意願與共識，對於公司財務與業務並未產生影響。\n \n \n 新店裕隆城 1080826工地空拍 裕隆組織大改造！納智捷董座陳國榮辭職 由裕隆總座姚振祥兼任\n ETtoday新聞雲 記者周康玉／台北報導 2019年09月12日 17:07\n \n 裕隆（2201）集團旗下自有品牌納智捷（LUXGEN）銷售表現始終未見起色，裕隆今（12）日證實，納智捷董事長、同時身兼華創車電董座的陳國榮已主動辭職，為品牌表現不佳負責。裕隆表示，已慰留陳國榮華創車電董座職務，未來納智捷董座由裕隆汽車總經理姚振祥兼任；裕隆強調，同意陳國榮辭董座是基於策略轉型考量，陳國榮仍為集團重要的決策委員。\n \n 裕隆表示，為了落實集團汽車價值鏈轉型策略及華創車電廠辦合一，集團組織及人事進行以下調整：\n 一. 設立「自有品牌暨開放平台整合專案室」，負責納智捷品牌及開放平台價值鏈研發、採購、製造、銷售之協調與整合。該專案室總監乙職由裕隆汽車總經理姚振祥兼任，副總監由華創車電左自生副總經理擔任。\n 二. 另為強化溝通及運作效率，「自有品牌暨開放平台整合專案室」直接向集團決策委員會報告。另，納智捷汽車董事長一職改由裕隆汽車總經理姚振祥兼任。\n 三. 華創車電總經理李俊忠辭任，改由副總經理黃振宏暫代華創車電總經理職務。\n 四. 華創車電董事長陳國榮請辭，已予慰留。\n 五. 集團總管理處與決策委員會即

In [118]:
with open('prompts/openai_prompts_42.json', 'w') as f:
    json.dump(prompts, f)

In [85]:
len(prompts)

87

In [86]:
len_p = [len(tokenizer(p['prompt'])['input_ids']) for p in prompts]

In [87]:
max(len_p)

3996

In [88]:
len_p.index(7698)

ValueError: 7698 is not in list

In [89]:
min(len_p)

3814

In [36]:
len(tdata)

87

In [37]:
len_p

[3991,
 3988,
 3939,
 3917,
 3851,
 3856,
 3945,
 3996,
 3920,
 3877,
 3934,
 3979,
 3978,
 3935,
 3929,
 3837,
 3991,
 3939,
 3992,
 3894,
 3915,
 3902,
 3928,
 3856,
 3975,
 3890,
 3981,
 3910,
 3922,
 3934,
 3913,
 3978,
 3967,
 3878,
 3884,
 3987,
 3936,
 3894,
 3572,
 3909,
 3959,
 3913,
 3850,
 3826,
 3873,
 3875,
 3936,
 3834,
 3835,
 3951,
 3929,
 3959,
 3589,
 3992,
 3968,
 3942,
 7698,
 3956,
 3802,
 3904,
 3874,
 3852,
 3751,
 3969,
 3877,
 3736,
 3931,
 3833,
 3939,
 3970,
 3980,
 3963,
 3850,
 3961,
 3910,
 3980,
 3944,
 3912,
 3870,
 3989,
 3853,
 3892,
 3946,
 3896,
 3570,
 3901,
 3795]

In [15]:
prompts[25]['prompt']

'post1 : 明後天漲停……夢到的 post2: Hotel部分似乎已經默默營業了\n 但是疫情影響 有多少人去住還未知\n \n 題外話\n 基泰這間公司的行銷真的很不行耶\n 管理部門不打算花點錢妝點一下新大樓嗎 > maximal potential profit (MPP)| 1# maximal loss (ML)| 0</s> post1 : 關於凱基蘋果分析師郭明錤此次的報料：「郭明錤還表示，市場對 2018 年下半年蘋果新款 iPhone 採用 Hybrid 鏡頭的期待過高。市場預期新款 iPhone 除了捨棄 Dotprojector 之 WLO（AMS 供應），改採 Hybrid（玻璃與塑膠混合）鏡頭（Hoya 與亞光的玻璃鏡片加上大立光與玉晶光的塑膠鏡片）之外，還將捨棄 Infrared camera 的 4P 鏡頭（大立光與玉晶光供應），改採 Hybrid 鏡頭（AAC 提供）。但基於凱基對 2018 年下半年新款 iPhone 將準時且順暢供貨，以及 TrueDepth camera 規格無顯著升級的假設，因此認為上述市場預期的發生機會不高。」\n 亞光被點名2018無法打入蘋果供應鍊，因蘋果擔心其產能不足，恐造成出貨不順。。\n \n 不知各位看法如何？ 像大立光因應鏡頭鏡片數不斷從4p,5p甚至到7p鏡片數的須求增加，甚至也能吃下其他大廠得鏡片須求，不就是有需求就大力擴廠，而獲利就節節高升。 亞光不就應付混合鏡頭1g4p或2g3p 的1至2片玻璃鏡片g的部分，這樣也沒辦法趕快擴廠增加產能嗎？\n 這個消息正確性很高嗎？ 近期亞光股價不振似乎回應了這則預測。 那其他次品牌大陸韓國和日本手機大廠有可能對亞光下訂單嗎？ post2: 11/14\n 「手機鏡頭方面，過去手機鏡頭大多使用塑膠鏡片，從早期三片、四片塑膠鏡片，到現在已經用到六片塑膠鏡片、甚至開始研發七片結構。雙鏡頭設計整合廣角鏡頭與長焦鏡頭、將塑膠鏡頭的功能推上新境界，但也意味塑膠鏡頭在照相功能的極限將至。為了提高變焦倍數，搭配玻璃鏡片的潛望式變焦鏡頭應運而生。\n \n 據悉，目前大陸手機大廠OPPO、華為對於五倍變焦潛望式鏡頭都有興趣，亞光亦積極投入五倍變焦手機潛望式鏡頭的設計與製造，現階段正與日本廠商（驅動馬達）及以色列廠商（影像處理軟體）共同合作開發，預計最快在今年底將

In [53]:
s = prompts[22]['prompt']

In [54]:
s

'post1 : 明後天漲停……夢到的 post2: Hotel部分似乎已經默默營業了\n 但是疫情影響 有多少人去住還未知\n \n 題外話\n 基泰這間公司的行銷真的很不行耶\n 管理部門不打算花點錢妝點一下新大樓嗎 > maximal potential profit (MPP)| 1# maximal loss (ML)| 0</s> post1 : 關於凱基蘋果分析師郭明錤此次的報料：「郭明錤還表示，市場對 2018 年下半年蘋果新款 iPhone 採用 Hybrid 鏡頭的期待過高。市場預期新款 iPhone 除了捨棄 Dotprojector 之 WLO（AMS 供應），改採 Hybrid（玻璃與塑膠混合）鏡頭（Hoya 與亞光的玻璃鏡片加上大立光與玉晶光的塑膠鏡片）之外，還將捨棄 Infrared camera 的 4P 鏡頭（大立光與玉晶光供應），改採 Hybrid 鏡頭（AAC 提供）。但基於凱基對 2018 年下半年新款 iPhone 將準時且順暢供貨，以及 TrueDepth camera 規格無顯著升級的假設，因此認為上述市場預期的發生機會不高。」\n 亞光被點名2018無法打入蘋果供應鍊，因蘋果擔心其產能不足，恐造成出貨不順。。\n \n 不知各位看法如何？ 像大立光因應鏡頭鏡片數不斷從4p,5p甚至到7p鏡片數的須求增加，甚至也能吃下其他大廠得鏡片須求，不就是有需求就大力擴廠，而獲利就節節高升。 亞光不就應付混合鏡頭1g4p或2g3p 的1至2片玻璃鏡片g的部分，這樣也沒辦法趕快擴廠增加產能嗎？\n 這個消息正確性很高嗎？ 近期亞光股價不振似乎回應了這則預測。 那其他次品牌大陸韓國和日本手機大廠有可能對亞光下訂單嗎？ post2: 11/14\n 「手機鏡頭方面，過去手機鏡頭大多使用塑膠鏡片，從早期三片、四片塑膠鏡片，到現在已經用到六片塑膠鏡片、甚至開始研發七片結構。雙鏡頭設計整合廣角鏡頭與長焦鏡頭、將塑膠鏡頭的功能推上新境界，但也意味塑膠鏡頭在照相功能的極限將至。為了提高變焦倍數，搭配玻璃鏡片的潛望式變焦鏡頭應運而生。\n \n 據悉，目前大陸手機大廠OPPO、華為對於五倍變焦潛望式鏡頭都有興趣，亞光亦積極投入五倍變焦手機潛望式鏡頭的設計與製造，現階段正與日本廠商（驅動馬達）及以色列廠商（影像處理軟體）共同合作開發，預計最快在今年底將